# Python Średniozaawansowany

#### Instrukcja instalacji VS Code <a class="anchor" id="install"></a>

 - Zainstaluj narzędzia [Python](https://www.python.org/downloads/).

 - Pobierz [VS Code](https://code.visualstudio.com/).

 - Zainstaluj rozszerzenie [Python](https://marketplace.visualstudio.com/items?itemName=ms-python.python).

 - Zainstaluj rozszerzenie [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

  - Zainstaluj rozszerzenie [Git Extension Pack](https://marketplace.visualstudio.com/items?itemName=donjayamanne.git-extension-pack).

#### Potrzebne importy

 - [matplotlib](https://matplotlib.org/) - wymagana instalacja
    * [pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)
 - [numpy](https://numpy.org/) - wymagana instalacja
 - [pandas](https://pandas.pydata.org/) - wymagana instalacja
 - os
 - [inspect](https://docs.python.org/3/library/inspect.html) - biblioteka zawierająca [funkcjonalność przydatną przy analizowaniu struktur klas](https://docs.python.org/3/library/inspect.html#classes-and-functions).
 - [abc](https://docs.python.org/3/library/abc.html) - biblioteka do abstrakcji klasowej


In [ ]:
from abc import ABC, abstractmethod
import traceback
import os
from inspect import getmro
from IPython.display import clear_output
from IPython.core.magic import register_line_magic, register_cell_magic
from IPython import get_ipython

In [ ]:
%pip install matplotlib
%pip install numpy
%pip install pandas
clear_output()

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

#### Przydatne definicje [magicznych komend](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

 * `%%captureErrors` - przechwytuje wszelkie błędy występujące w komórce i drukuje je jako zwykły tekst. Kolorowanie wzorowane na [wątku](https://stackoverflow.com/questions/61139640/is-it-possible-to-print-traceback-format-exc-in-color-using-colored-tracebac) ze StackOverFlow.

In [ ]:
import traceback
from pygments import formatters, highlight, lexers

@register_cell_magic('captureErrors')
def captureErrors(line, cell):
    try:
        exec(cell)
    except:
        tb_text = "".join(traceback.format_exc())

        lexer = lexers.get_lexer_by_name("pytb", stripall=True)
        formatter = formatters.get_formatter_by_name("terminal256")
        tb_colored = highlight(tb_text, lexer, formatter)

        print(tb_colored)


## Wyjątki

### Problem:

W przypadku źle napisanego kodu, nieoczekiwanych przypadków brzegowych lub awarii systemu, program może natrafić na osobliwość, która uniemożliwia dalsze działanie i wymaga albo przerwanie wykonania, albo naprawienie stanu w trakcie wykonania. W jaki sposób przekazać informację o błędzie oraz w jaki sposób obsłużyć taki błąd?

### [Wyjątki](https://en.wikipedia.org/wiki/Exception_handling) są zdarzeniami, które pojawiają się podczas wykonania programu i przerywają jego normalny przebieg. Technicznie są to obiekty reprezentujące dany błąd.

In [ ]:
%%captureErrors
a = 5.0
b = 0.0
print(a/b)

##### Wszystkie wbudowane, nie opuszczające systemu [wyjątki](https://docs.python.org/3/library/exceptions.html) dziedziczą po klasie `Exception`. Wyjątki tworzone przez programistę również powinny dziedziczyć po tej klasie.

##### Najczęściej spotykane wyjątki to

 - `AssertionError` - niespełnienie warunku operacji [assert](https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement)
 - `AttributeError` - odwołanie do nieistniejącego atrybutu
 - `IOError` - błąd operacji wejścia/wyjścia
 - `IndexError` - odwołanie do kolekcji za pomocą indeksu poza zakresem
 - `ImportError` - problem z załadowaniem modułu
 - `KeyError` - odwołanie do słownika nieistniejącym kluczem
 - `ValueError` - przekazanie do funkcji argumentu o niepoprawnej wartości
 - `ZeroDivisionError` - dzielenie przez zero

### Obsługa wyjątków

##### Wyjątki obsługujemy za pomocą słów kluczowych:
 - `try` - poprzedza obszar sprawdzany pod kątem wyrzucania wyjątków,
 - `except` - poprzedza obszar obsługi wyłapanego wyjątku,
 - `finally` - poprzedza obszar, który jest wykonywany pod koniec obsługi, niezależnie od tego, czy wyjątek został wyrzucony czy też nie
 - `raise` - wyrzuca zadany wyjątek

In [ ]:
for i in [0.5, 0.3, 0]:
    try:
        result = 10/i
    except:
        result = 10000000
    print(i, " ", result)

In [ ]:
def root4(x):
    return x ** 0.25

x = "Some dummy text"

try:
    print(root4(x))
except:
    print("Hey! I cannot take a root of that")

In [ ]:
%%captureErrors
def f():
    for i in [0.5, 0.3, 0.001, 2.0]:
        try:
            result = 10/(max(0.1,i) - 0.1)
        except:
            result = 10000000
            raise ValueError("Value lower or equal to 0.1")
        finally:
            print(i, " ", result)

def g():
    def h():
        f()
    h()
g()

##### Do każdego bloku `try` możemy przypisać wiele bloków `except`.

In [ ]:
l = [5, 0, 6, 2]

for i in range(5):
    try:
        a = 1/l[i]
        print(a)
        assert a >= 0.2
    except KeyError:
        print("Wrong key")
    except (ZeroDivisionError):
        print("Oopsie, whoopsie. You tried to divide by zero")
    except (IndexError, KeyError):
        print("I went too far")
    except:
        print("What did just happened here?")

##### Klauzulę `finally` zazwyczaj używamy, gdy po wykonaniu zadania musimy zwolnić pewne zasoby, niezależnie, czy procedura powiodła się czy nie. Na przykład, obsługa pliku.

In [ ]:
%%captureErrors
dummy_file_name = "dummy.file"

file = None
try:
    file = open(dummy_file_name, "r")
    print(file.read())
finally:
    if file:
        file.close()
    if os.path.exists(dummy_file_name):
        os.remove(dummy_file_name)


### Manager kontekstu

##### Zamiast tego możemy użyć wyrażenia [with](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) zwanego managerem kontekstu, które obsługuje obiekty posiadające metody `__enter__` i `__exit__`

W poniższym przykładzie [open](https://docs.python.org/3/library/functions.html#open) tworzy uchwyt do pliku. Za pomocą menagera kontekstu zapisujemy uchwyt do zmiennej `file` i zabezpieczamy się przed ewentualnymi błędami w działaniu programu. Jeżeli dojdzie do przerwania, manager kontekstu zamknie uchwyt zwalniając plik.

In [ ]:
%%captureErrors

dummy_file_name = "dummy.file"

with open(dummy_file_name, "w") as file:
    file.write("Somedummytextjusttofillthefilewithsomecontent")

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

with open(dummy_file_name, "r") as file:
    print(file.read())

In [ ]:
%%captureErrors

class MyContextManager:
    def __enter__(self):
        print("enter")
        return 5

    def __exit__(self, t, value, tb):
        print("exit")
        print(t, value, tb)

with MyContextManager() as v:
    print("Hi", v)

with MyContextManager() as v:
    raise Exception("Ops")

Podobną funkcjonalność można osiągnąć również z pomocą dekoratora [@contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager)

In [ ]:
%%captureErrors

from contextlib import contextmanager

dummy_file_name = "dummy.file"

class ReadWriteFile:
    def __init__(self, address) -> None:
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

@contextmanager
def autodestructingFile(address):
    file = ReadWriteFile(address)
    try:
        yield file
    finally:
        file.reader.close()
        file.writer.close()
        if os.path.exists(address):
            os.remove(address)


with autodestructingFile(dummy_file_name) as file:
    file.write("somedummytext")
    print(file.read())

    raise Exception("some dummy exception")
    print("message ignored")

### [Rzucanie wyjątków](https://docs.python.org/3/tutorial/errors.html#raising-exceptions)

##### W celu rzucenia wyjątku wykorzystujemy wyrażenie `raise`.

In [ ]:
%%captureErrors
raise Exception("My dummy exception")

###### <font color="gray">W większości języków odpowiednikiem `raise` jest `throw`</font>

### Tworzenie własnych wyjątków

In [ ]:
%%captureErrors
class CustomException1(Exception):
    pass

raise CustomException1("Some custom exception")

In [ ]:
%%captureErrors

class CustomException2(Exception):
    def __init__(self) -> None:
        super().__init__("My hardcoded message")

    def sing(self):
        print("In the marry month of June")

try:
    raise CustomException2()
except CustomException2 as e:
    e.sing()
    raise e

###### <font color='red'>Uwaga, niektóre wyjątki zabijają jądro Jupyter. Poniższy przykład ma na celu wywołania jednego z takich wyjątków. Po śmierci jądra cały kontekst wywołania zostanie stracony i wszelkie zmienne i importy będą musiały zostać wprowadzone na nowo.</font>

Dla jakiego `n` poniższy kod może nigdy się nie zakończyć? Czemu? Czemu jednak się kończy? Jakim błędem się kończy? (wywołaj poza środowiskiem `.ipynb` żeby uzyskać treść błędu) Co to znaczy?

In [ ]:
def fact(n):
    if n == 0:
        return 1
    else:
        return fact(n - 1) * n


 1. <font size="1" color="gray">Dowolna liczba nie będąca liczbą naturalną</font>
 2. <font size="1" color="gray">Funkcja będzie wywoływała siebie samą w nieskończoność. To tak zwana [rekurencja](https://en.wikipedia.org/wiki/Recursion).</font>
 3. <font size="1" color="gray">Python przerywa samodzielnie zbyt głęboko schodzące rekursje.</font>
 4. <font size="1" color="gray">Przekroczono dopuszczalną głębokość rekurencji. Przerywa to program zanim zostanie wyrzucony systemowy błąd `stackoverflow`.</font>



## [Programowanie obiektowe](https://en.wikipedia.org/wiki/Object-oriented_programming)

### Problem:

Program da się zazwyczaj podzielić na małe bloczki, które komunikują się ze sobą nawzajem. Często bloczki te są powtarzalne i da się je zdefiniować wspólnie jedną definicją. W jaki sposób stworzyć strukturę hierarchii i zależności? W jaki sposób przygotować możliwość zastąpienia niektórych dotychczasowych bloczków przez nowe warianty? W jaki sposób uogólnić formułę bloczku tak, by zdefiniować bazę wielu bloczków na raz? W jaki sposób napisać kod, w którym widoczne jest to, co bloczek wykorzystuje na własną potrzebę a co może być wykorzystywane przez inne bloczki?

##### Podstawowe koncepcje to

 - [dziedziczenie](https://en.wikipedia.org/wiki/Inheritance_(object-oriented_programming)),
 - [kompozycja](https://en.wikipedia.org/wiki/Object_composition),
 - [abstrakcja](https://en.wikipedia.org/wiki/Abstraction_(computer_science)),
 - [hermetyzacja](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)),
 - [polimorfizm](https://en.wikipedia.org/wiki/Polymorphism_(computer_science)).

### Dziedziczenie

Jeżeli pewna klasa posiada funkcjonalność, która jest nam potrzebna, ale chcielibyśmy jeszcze dodać coś od siebie, możemy rozszerzyć klasę tworząc nową, która dziedziczy funkcjonalności klasy bazowej.

In [ ]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name

    def eat(self, food):
        print(self.name, ": Nom, nom, nom. Tasty", food)

class Bird(Animal):
    def fly(self):
        print(self.name, ": I can fly")

animal = Animal("Creature")
animal.eat("other animal")

bird = Bird("Tweety")
bird.eat("grain")
bird.fly()

Możemy sprawdzić pochodzenie naszej klasy za pomocą funkcji [getmro](https://docs.python.org/3/library/inspect.html#inspect.getmro) z pakietu `inspect`.

In [ ]:
from inspect import getmro

print(getmro(type(bird)))
print(getmro(type(getmro)))
print(getmro(AttributeError))
print(getmro(type(5.0)))

Możemy też dziedziczyć po wielu klasach przejmując atrybuty każdej z nich.

In [ ]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name
        print("I am an animal")

    def eat(self, food):
        print(self.name, ": Nom, nom, nom. Tasty", food)

class Oviparous(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am an oviparous")

    def bearEgg(self):
        print(self.name, ": plop")

    def breath(self):
        print(self.name, ": wheez")

class Mammal(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a mammal")

    def giveMilk(self):
        print(self.name, ": blurp")

    def breath(self):
        print(self.name, ": phew")

class Platypus(Mammal, Oviparous):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a platypus")

    def sting(self):
        print(self.name, ": sting")

perry = Platypus("Perry")

perry.bearEgg()
perry.giveMilk()
perry.eat("algae")
perry.breath()

print(getmro(Platypus))

Zauważmy, że powyższa klasa rozwiązuje dziedziczenie wpierw rozpatrując klasę `Mammal`. Oznacza to, że jeśli klasy `Mammal` i `Oviparous` będą miały takie samo pole, `Platypus` przejmie tylko pole od `Mammal`.

### Kompozycja

Zamiast dziedziczyć wszystkie atrybuty możemy wkomponować jedną klasę w drugą tworząc obiekt tej pierwszej jako pole drugiej. Następnie pole to możemy wykorzystać do implementacja własnych metod.

In [ ]:
class Staff:
    def use(self):
        pass

class Mage():
    def __init__(self, name: str, staff: Staff) -> None:
        self.name = name
        self.staff = staff

    def castSpell(self):
        return f'{self.name}: {self.staff.use()}'


class FireWand(Staff):
    def use(self) -> str:
        return "Fireball!"

class FrostWand(Staff):
    def use(self) -> str:
        return "Brrr"

mage1 = Mage("Master", FireWand())
mage2 = Mage("Timothy", FrostWand())

print(mage1.castSpell())
print(mage2.castSpell())

In [ ]:
from __future__ import annotations

class Hero():
    def __init__(self, name: str) -> None:
        self.name = name

    def introduce(self) -> str:
        return f"I am {self.name}"

class Mage(Hero):
    def __init__(self, name: str, staff: Staff) -> None:
        super().__init__(name)
        self.staff = staff

    def equipStaff(self, staff: Staff):
        self.staff = staff

    def castSpell(self):
        if self.staff:
            return self.staff.use()
        else:
            return None

class Staff:
    def use(self):
        pass

    def setOwner(self, mage: Mage) -> None:
        self.mage = mage

class FireWand(Staff):
    def use(self) -> str:
        return "Fireball!"

class FrostWand(Staff):
    def use(self) -> str:
        return "Brrr"

mage1 = Mage("Master", FireWand())
mage2 = Mage("Timothy", FrostWand())

print(mage1.introduce(), mage1.castSpell())
print(mage2.introduce(), mage2.castSpell())

Zarówno Tymek jak i Mistrz potrafią czarować, potrzebują jednak do tego różdżek. Czarodziej nie musi wiedzieć, jak zostaje wykonany czar, potrzebuje tylko wiedzieć, jak korzystać z różdżki. W ten sposób rozdzielamy implementację obu klas i obiekt wkomponowany może być w dowolnym momencie zmieniony na nową implementację nie zmieniając implementacji klasy komponującej.

### Abstrakcja

Czasami chcemy stworzyć wzór klasy bez implementowania poszczególnych jej elementów. Przkładowo, chcę stworzyć klasę `Collection`, która pozwoli użytkownikowi na dostęp do elementów, ale pozwoli również zaimplementować różne kontenery przechowujące elementy.

Nie jest to podstawowy koncept w Pythonie ze względu na brak [silnej typizacji](https://en.wikipedia.org/wiki/Strong_and_weak_typing), stąd musimy wykorzystać do tego pakiet `abc`.

###### Python nie posiada silnej typizacji stąd koncept abstrakcji jest czystym formalizmem. W językach z silną typizacją zawsze musimy podać, jakiego typu dane przyjmuje funkcja. Jeżeli nie chcemy podawać gotowej implementacji od razu przy tworzeniu funkcji, możemy stworzyć klasę-wydmuszkę, która będzie tylko szablonem pod klasy obiektów przekazywanych do owej funkcji.

In [ ]:
class Collection:
    def __init__(self, elements) -> None:
        pass

    def getNext(self):
        pass

def iterate(collection: Collection, n: int, f):
    for i in range(n):
        f(collection.getNext())

c = Collection([1, 2, 4])
print(c.getNext())

class SingleElementCollection(Collection):
    def __init__(self, elements) -> None:
        super().__init__(elements)
        self.element = elements

    def getNext(self):
        return self.element

c = SingleElementCollection([1, 2, 4])

class LoopCollection(Collection):
    def __init__(self, elements) -> None:
        super().__init__(elements)
        self.elemnts = elements
        self.index = 0

    def getNext(self):
        element = self.elemnts[self.index]
        self.index = (self.index + 1) % len(self.elemnts)
        return element

c = LoopCollection([1, 2, 4])

iterate(c, 10, print)

W powyższym kodzie klasa `Collection` jest w pełni poprawną klasą i można stworzyć jej obiekt. Klasa abstrakcyjna nie powinna dawać możliwości stworzenia swojej instancji. Jest to wyłącznie wydmuszka, która służy jako wzorzec do stworzenia poprawnej implementacji.

Żeby poinformować, że klasa ta jest wyłącznie wydmuszką stosujemy pakiet [abc](https://docs.python.org/3/library/abc.html). W kodzie poniżej klasa `Collection` dziedziczy po klasie `ABC` i posiada pola z dekoratorem [@abstractmethod](https://docs.python.org/3/library/abc.html#abc.abstractmethod). Z tego powodu nie można stworzyć instancji tej klasy a jedynie instancje klas dziedziczących, które implementują te pola.

In [ ]:
from abc import ABC, abstractmethod

class Collection(ABC):
    @abstractmethod
    def __init__(self, elements) -> None:
        pass

    @abstractmethod
    def getNext(self):
        pass

class SingleElement(Collection):
    def __init__(self, elements) -> None:
        self.element = elements

    def getNext(self):
        return self.element

class LoopCollection(Collection):
    def __init__(self, elements) -> None:
        self.elements = elements
        self.i = 0

    def getNext(self):
        element = self.elements[self.i]
        self.i %= len(self.elements)
        return element

c = LoopCollection([1, 2, 4])

def iterate(collection: Collection, n: int, f):
    for i in range(n):
        f(collection.getNext())

iterate(c, 10, print)

In [ ]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def __init__(self, name: str) -> None:
        pass

class Bird(Animal, ABC):
    @abstractmethod
    def tweet(self):
        pass

class Robin(Bird):
    def __init__(self, name: str) -> None:
        self.name = name

    def tweet(self):
        return f"{self.name}: tweet tweet"

class Swan(Bird):
    def __init__(self, name: str) -> None:
        self.name = name[::-1]

    def tweet(self):
        return f"{self.name}: hsssss"

def feed(bird: Bird):
    print(bird.tweet())

robin = Robin("Robyn")
swan = Swan("ducklinG")
feed(robin)
feed(swan)

### Hermetyzacja

Niektóre pola nie powinny być dostępne z zewnątrz. Jeżeli nasza klasa wykonuje jakąś operację na własnych parametrach możemy chcieć chronić te parametry przed zmianą z zewnątrz lub przez klasy dziedziczące.

Pola [prywatne](https://en.wikipedia.org/wiki/Access_modifiers) są dostępne tylko dla klasy zawierającej owe pole. Klasy dziedziczące nie mają do tego pola dostępu. Podejście takie nazywamy hermetyzacją. Python nie posiada mechanizmu hermetyzacji i pola są zawsze dostępne. Jeżeli chcemy zaznaczyć, że pole jest zamierzone jako chronione zaczynamy jego nazwę od podkreślnika (`_`), jeżeli zaś chcemy, żeby było traktowane jako prywatne zaczynamy jego nazwę od podwójnego podkreślnika (`__`).

In [ ]:
class ProtectedDummy:
    def __init__(self) -> None:
        self._counter = 0

    def add(self):
        self._counter += 1

    def getCounter(self):
        return self._counter

class ProtectedChild(ProtectedDummy):
    def add(self):
        self._counter += 3

dummy = ProtectedDummy()
dummy.add()
dummy.add()
dummy.add()
dummy._counter = -3
dummy.add()
dummy.add()
dummy.add()
dummy.getCounter()

Pola prywatne wciąż są osiągalne z zewnątrz jako pola o nazwie `_<CLASS_NAME><FIELD_NAME>`.

In [ ]:
class PrivateDummy:
    def __init__(self) -> None:
        self.__private = 0

    def add(self):
        self.__private += 1

    def counter(self):
        return self.__private

dummy = PrivateDummy()
dir(dummy)

dummy.add()
dummy.add()
dummy.add()
dummy._PrivateDummy__private = -3
dummy.add()
dummy.add()
dummy.add()
print(dummy.counter())

### Polimorfizm

Python nie implementuje pełnego polimorfizmu. W językach silnie typowanych możemy zdefiniować dwie funkcje o tej samej nazwie, przyjmujące jednak parametry różnego typu.

In [ ]:
def f(a: int):
    print("This is an integer")

def f(b: str):
    print("This is a string")

f(1)
f("somedummytext")

Python nie daje takiej możliwości i zostanie wywołana wyłącznie funkcja ostatnio zdefiniowana. Nie istnieje w nim również możliwość tworzenia funkcji o tej samej nazwie ale innej liczbie parametrów.

In [ ]:
%%captureErrors
def f(a):
    print("I have got a single value")

def f(a, b):
    print("I have got two values")

f(1, 2)
f(3)

Jedyne postacie polimorfizmu w Pythonie to funkcje wywołujące te same operacje na obiektach różnych typów, tak zwany [duck typing](https://en.wikipedia.org/wiki/Duck_typing).

In [ ]:
%%captureErrors
class Duck:
    def swim(self):
        print("Duck swimming")

    def fly(self):
        print("Duck flying")

class Whale:
    def swim(self):
        print("Whale swimming")

for animal in [Duck(), Whale()]:
    animal.swim()
    animal.fly()

### [Dataclass](https://docs.python.org/3/library/dataclasses.html)

Aby nie pisać za każdym razem konstruktora (`__init__`), reprezentacji tekstowej (`__repr__`) oraz funkcjonalności porównania (`__eq__`) możemy użyć dekoratora `@dataclass`. Zapisujemy pola takiej klasy pisząc ich nazwy z typami po dwukropku.

Co typowe dla Pythona, zadane typy są wyłącznie informacyjne i nic nie stoi na przeszkodzie użyca innych typów.

In [ ]:
from dataclasses import dataclass

@dataclass
class Dummy:
    a: int
    b: str
    c: list

dummy = Dummy(1, 2, 3)
dummy2 = Dummy(1, 2, 3)
print(dummy)
print(dummy == dummy2)
print(dummy.a, dummy.b, dummy.c)

Dektorator `@dataclass` udostępnia również możliwość tworzenie innej funkcjonalności:

 * operatory porównania (`<`, `>`, `<=` i `>=`)

In [ ]:
@dataclass(order=True)
class Pair:
    a: int
    b: float

Pair(1, 2.0) < Pair(2, 1.0)

 * [hashowanie](https://en.wikipedia.org/wiki/Hash_function) bez gwarancji niezmienności

In [ ]:
%%captureErrors
from dataclasses import field

@dataclass
class Nonhashable:
    l: int
    a: int
    b: int

@dataclass(unsafe_hash=True)
class Hashable:
    l: list = field(hash=False)
    a: int
    b: int

n = Nonhashable(0, 1, 2)
h = Hashable([1, 2, 3], 1, 2)
print(hash(h))
h.l = 3
print(hash(h))
{n}

 * niemutowalność pól

In [ ]:
%%captureErrors

@dataclass(frozen=True)
class FrozenClass:
    a: int
    b: float

f = FrozenClass(2, 4.5)
print(f)
print(f.a, f.b)
f.a = 4

Dlaczego `unsafe_hash` jest odradzane w użyciu? Może to prowadzić do nieprzewidzianych zachowań przy strukturach bazujących na hashu.

In [ ]:
@dataclass(unsafe_hash=True)
class Trine:
    a: int
    b: int
    c: int

t1 = Trine(1, 2, 3)
t2 = Trine(2, 5, 3)

s = {t1, t2}
print(s)
s.add(t1)
print(s)
print(t1 in s)
t1.a = 2
print(t1 in s)
s.add(t1)
print(s)

### Kopia płytka a głęboka

Kopię listy możemy wykonać za pomocą konstruktora `list`, który tworzy nową listę na podstawie zadanych elementów. Kopia taka nie kopiuje jednak elementów w liście a tworzy referencje do dokładnie tych samych obiektów. Tego typu kopię nazywamy kopią płytką.

In [ ]:
@dataclass
class Container:
    field: int

l1 = [Container(1), Container(2), Container(3)]
l2 = list(l1)
l2[0].field = 2
l2.append(l2[0])
print(l2)
l2[3].field = 3
print(l2)
print(l1)

W opozycji do płytkiej kopii, kopia głęboka kopiuje rekurencyjnie również obiekty wewnątrz. Do tej operacji wykorzystujemy [deepcopy](https://docs.python.org/3/library/copy.html#copy.deepcopy) z biblioteki [copy](https://docs.python.org/3/library/copy.html).

In [ ]:
from copy import deepcopy

@dataclass
class Container:
    field: int

l1 = [Container(1), Container(2), Container(3)]
l2 = deepcopy(l1)
l2[0].field = 2
print(l1)

c1 = Container(2)
c2 = Container(c1)
c3 = deepcopy(c2)
c3.field.field = 1
print(c2, c3)

Operacja `deepcopy` zapamiętuje już skopiowane pola, dzięki czemu nie tworzy na jednego elementu wielokrotnie jeżeli odniesienia do niego występują w wielu miejscach.

In [ ]:
c1 = Container(2)
c2 = Container(c1)
c1.field = c2

c3 = deepcopy(c1)
c3.field.field.field = 3
c3

In [ ]:
c1 = Container(2)
c2 = Container([c1, c1, c1])
c3 = deepcopy(c2)
c3.field[0].field = 3
print(c3)

## Serializacja

### Problem:

Dane, które przetwarzamy w programie mogą być potrzebne przy ponownym uruchomieniu programu lub do działania innych programów. W jaki sposób możemy zapisać te dane na dysku? W jaki sposób możemy je odczytać? W jaki sposób przygotować je do transferu pomiędzy maszynami?

[Serializacją](https://en.wikipedia.org/wiki/Serialization) nazywamy proces zapisu i odczytu danych do postaci możliwej do przechowywania na dysku. Rozróżniamy dwa rodzaje formatu danych:
 - [binarny](https://en.wikipedia.org/wiki/Binary_file) - dane przechowywane są w kompaktowej formie analogicznej do formatu w pamięci podręcznej. Format mało czytelny dla człowieka i trudny do odtworzenia, jeżeli nie posiadamy informacji o kodowaniu.
 - [tekstowy](https://en.wikipedia.org/wiki/Text_file) - dane zapisane są w postaci czytelnego tekstu. Format ten zajmuje o wiele więcej miejsca, jest jednak prosty do odczytania i nie wymaga żadnych informacji o formatowaniu.

Prostym przykładem serializacji jest zapis wbudowaną funkcjonalnością `write`. Operacja ta przyjmuje wyłącznie tekst, dane w ten sposób zapisane są zawsze danymi tekstowymi. Oznacza to, że dane nietekstowe są trudne w odczycie z takiego formatu.

In [ ]:
data = [1, 2, 3]

dummy_file_name = "dummy.txt"

with open(dummy_file_name, "w") as writer:
    writer.write(str(data))

with open(dummy_file_name, "r") as reader:
    l = reader.read()

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

list(l)

Serializacja do pliku binarnego w Pythonie odbywa się z pomocą modułów [marshal](https://en.wikipedia.org/wiki/Text_file) (wykorzystywany do operowania na pseudo-skompilowanym kodzie Pythona, niezalecane do innych zastosowań), [pickle](https://docs.python.org/3/library/pickle.html#module-pickle) czy [shelve](https://docs.python.org/3/library/shelve.html#module-shelve).

###### Moduły te nie wykonują samodzielnie operacji zapisu i odczytu a wykorzystują zadane im obiekty buforowe. Pozwala to, na przykład, na czytanie i zapisywanie danych na serwer lub korzystanie z wejścia/wyjścia do konsoli użytkownika lub innych narzędzi.

In [ ]:
import pickle

data = [1, 2, 3]

dummy_file_name = "dummy.txt"

with open(dummy_file_name, "wb") as writer:
    pickle.dump(data, writer)

with open(dummy_file_name, "rb") as reader:
    l = pickle.load(reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l[1]

Serializacja tekstowa może odbywać się na wiele sposobów. Jednym z częstszych jest wykorzystanie tak zwanej JavaScript Object Notation lub [JSON](https://en.wikipedia.org/wiki/JSON). W Pythonie wykorzystujemy do tego moduł [json](https://docs.python.org/3/library/json.html).

In [ ]:
import json

dummy_file_name = "dummy.json"

data = {
    "capitals":
    {
        "Poland": "Warsaw",
        "Sri Lanka": "Sri Jayawardenepura Kotte",
        "Czechia": "Prague"
    },
    "numbers": [1, 2, 3, 4],
    "number": 15
}

with open(dummy_file_name, "w") as writer:
    json.dump(data, writer)

with open(dummy_file_name, "r") as reader:
    l = json.load(reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l["number"]


###### Podobnym w strukturze formatej jest [XML](https://en.wikipedia.org/wiki/XML), do którego wykorzystujemy pakiet [xml.etree.ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html).

Formaty `JSON` i `XML` zapisują dane w formacie drzewiastym jako słowniki zagnieżdżone, to jest, są słownikami zawierającymi wartości, które tekstem, listą lub słownikiem. Inną formą zapisu danych jest zapis w postaci tablicy dwuwymiarowej. Taką formę uzyskujemy na przykład formatem [csv](https://en.wikipedia.org/wiki/Comma-separated_values). Serializujemy dane w tym formacie z pomocą modułu [csv](https://docs.python.org/3/library/csv.html).

In [ ]:
dummy_file_name = "pracownicy.csv"

data = [['pracownik', 'pensja'],
    ['Jan Kowalski', '2500'],
    ['Joanna Dolegiewicz', '7500'],
    ['Katarzyna Kot', '10000']]

import csv

with open(dummy_file_name, 'w', newline='') as writer:
    csv_writer = csv.writer(writer)
    csv_writer.writerows(data)

with open(dummy_file_name, "r") as reader:
    csv_reader = csv.reader(reader)
    l = list(csv_reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l

Wczytywanie z pomocą modułu `csv` tworzy tablicę tablic, która może być problematyczna w operowaniu. Pliki, z których wczytujemy mogą mieć również pewne niestandardowe nagłówki i stópki a także mieć niestandardowe rozdzielanie danych. Dla lepszego operowania na większych zbiorach danych zaleca się zewnętrzny moduł [pandas](https://pandas.pydata.org/).

In [ ]:
import pandas as pd

data = pd.read_csv("./samples/SBE39plus_03907421_2022_10_26.asc", sep=",", skiprows=11, header=None)
display(data)
display(data[(data[0]>1)&(data[0]<15)])
data[4] = data[1] ** 2.0 + 12
display(data)
display(data.sort_values(by=[0]))


## Dekoratory

### Problem:

Mając dostępną funkcjonalność możemy chcieć ją rozwinąć. W jaki sposób możemy rozszerzyć działanie funkcji zachowując jej wywołania w dalszym kodzie? W jaki sposób stworzyć funkcjonalność, która rozszerzy dowolną funkcję o pewien powtarzalny zestaw instrukcji?

Dekorator w Pythonie to nakładka (formalnie [funkcjonał](https://en.wikipedia.org/wiki/Functional_(mathematics))), którą możemy nałożyć na funkcję. Pozwala na stworzenie kontekstu wywołania dla definiowanej funkcji. Dekorator w ten sposób jest funkcją, która zwraca funkcję, która ma zostać wykonana zamiast funkcji dekorowanej.

In [ ]:
def log_usage(func):
    def wrapper():
        print("Function execution called")
        result = func()
        print(f"{result} obtained as the result of the call")
        return result
    return wrapper

@log_usage
def dummyFunction():
    return 0

dummyFunction()
pass

Dekorator może również przyjmować argumenty, które zdefiniują jego zachowanie. W tym wypadku musimy otoczyć definicję dekoratora nakładką, która zwróci sam dekorator na podstawie zadanych argumentów.

In [ ]:
def log_usage(name, greeting):
    def dec(func):
        def wrapper():
            print(f"{name}: {greeting}")
            print(f"{name}: Function execution called")
            result = func()
            print(f"{name}: {result} obtained as the result of the call")
            return result
        return wrapper
    return dec

@log_usage("Dummy logger", "Dummy hello")
def dummyFunction():
    return 0

dummyFunction()

Gdy funkcja dekorowana przyjmuje argumenty nie możemy wykorzystać dekoratora wywołującego ją bez nich.

In [ ]:
%%captureErrors
def decorated(func):
    def wrapper():
        func()
    return wrapper

@decorated
def func(a):
    print(a)

func(2)

Zamiast tego powinniśmy użyć przechwycenia listy argumentów z pomocą `*args` i `**kwargs`.

In [ ]:
def decorated(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
    return wrapper

@decorated
def func(a, b):
    print(a)

func(2, 3)

## [Programowanie funkcjonałowe](https://en.wikipedia.org/wiki/Functional_programming)

### Problem:

Niektóre operacje wymagają dodatkowych instrukcji dla danych lub kontekstu wywołania. Przykładowo sortowanie wymaga operatora porównania. W jaki sposób przekazać instrukcję do wykonania bez definiowania mnóstwa metadanych dookoła?

[Wyrażeniem lambda](https://en.wikipedia.org/wiki/Anonymous_function) lub funkcją anonimową nazywamy definicję funkcji jako wartości. Innymi słowy, zamiast definiować funkcję a następnie przekazać ją jako argument do innej funkcji, możemy zdefiniować ją od razu jako ów argument. Nazwa ta wywodzi się z tak zwanego [rachunku $\lambda$](https://en.wikipedia.org/wiki/Lambda_calculus) będącego bazą [języków funkcjonałowych](https://en.wikipedia.org/wiki/Functional_programming).

Jeżeli chcemy posortować listę par po drugim elemencie, musimy podać funkcji [sorted](https://docs.python.org/3/howto/sorting.html#sorting-basics) argument `key` będący funkcją zwracającą klucz, po którym sortujemy.

In [ ]:
def second(pair):
    return pair[1]

l = [(1, 2), (2, 1), (5, 1), (12, 3), (12, 0)]
sorted(l, key=second)

Możemy taką funkcję od razu zdefiniować jako argument za pomocą słowa kluczowego `lambda`.

In [ ]:
l = [(1, 2), (2, 1), (5, 1), (12, 3), (12, 0)]
sorted(l, key=lambda pair: pair[1])

W celu zdefiniowania funkcji anonimowej piszemy `lambda`, następnie listę parametrów i na koniec, po dwukropku (`:`) samo ciało funkcji. Przykładowo, poniższe funkcje działają identycznie.

In [ ]:
def printAndReturnSum(a, b, c):
    print(f"I received {a} as a, {b} as b and {c} as c")
    return a + b + c

lambdaPrintAndReturnSum = lambda a, b, c: print(
    f"I received {a} as a, {b} as b and {c} as c") or a+b+c

printAndReturnSum(1, 2, 3) == lambdaPrintAndReturnSum(1, 2, 3)

<font color="red">!!!</font> Wyrażenia lambda pozwalają na wykonanie tylko jednej operacji, w powyższym przykładzie pokazane jest jednak, że możemy wykonać takich operacji kilka. Operator `or` pierwszą wartość jeżeli jest niefałszywa, drugą wartość w przeciwnym wypadku. `print` nie zwraca niczego, stąd jego wartość to `None`, który jest wartością fałszywą. Stąd zwracamy drugi argument operatora `or`.

<font color="red" size="5">Nie powinno się z tego korzystać w Pythonie</font>

Jeżeli potrzebujemy wykonać kilka operacji, powinniśmy zdefiniować zwyczajną funkcję. Powyższy przykład z `or` jest trikiem optymalizacyjnym w niektórych językach, w Pythonie jednak jest niepoprawnym wzorcem.

### Typowe zastosowania $\lambda$-wyrażeń: map i filter.

Często $\lambda$-wyrażenia wykorzystujemy do operacji na tablicach. Przykładowo, jeżeli chcemy posortować listę par po drugich elementach, podajemy do funkcji `sorted` jako parametr `key` albo funkcję pozyskującą ten element, albo równoważną $\lambda$.

In [ ]:
def takeSecond(e):
    return e[1]

l = [(3, 1), (2, 2), (2, 4), (4, 4), (1, 4), (0, 0), (3241, 1), (0, 1)]
print(sorted(l))
print(sorted(l, key=takeSecond))
print(sorted(l, key=lambda e: e[1]))

Analogicznie wykorzystujemy $\lambda$-wyrażenia z operacjami [max](https://docs.python.org/3/library/functions.html#max) i [min](https://docs.python.org/3/library/functions.html#min).

In [ ]:
def takeSecond(e):
    return e[1]

l = [(3, 1), (2, 2), (2, 4), (4, 4), (1, 4), (0, 0), (3241, 1), (0, 1)]
print(max(l, key=takeSecond))
print(max(l, key=lambda e: e[1]))
print(min(l, key=takeSecond))
print(min(l, key=lambda e: e[1]))

###### <font color="gray">Zarówno max jak i min przechodzą po wszystkich elementach i sprawdzają, czy klucz jest odpowiednio większy lub mniejszy od dotąd największego/najmniejszego znalezionego. Stąd, mimo, że występuje wiele elementów o kluczu 4, to jako maximum bierzemy tylko pierwszy przypadek.</font>

Podobnie możemy zaaplikować pewną funkcję do elementów tablicy. Operację taką nazywamy [mapowaniem](https://en.wikipedia.org/wiki/Map_(mathematics)).

In [ ]:
l = range(10)

def doubleAndAdd1(n):
    return n * 2 + 1

display([doubleAndAdd1(x) for x in l])
display([2 * x + 1 for x in l])
display(map(doubleAndAdd1, l))
display(map(lambda x: 2 * x + 1, l))

Warto zauważyć, że operacja [map](https://docs.python.org/3/library/functions.html#map) nie zwraca listy a objekt typu `map`. O różnicy, w tym o leniwości i gorliwości algorytmu, będzie więcej w dalszej części notatnika.

Warto zauważyć, że wykonanie operacji `map` czy pętli można zastąpić znacznie krótszymi i szybszymi operacjami na [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html).

In [ ]:
import numpy as np

l = np.arange(100000)

display(l)
display(l * 2 + 1)
display(l ** 0.15 - 12 * l)

$\lambda$-wyrażenia są również wykorzystywane przy filtrowaniu danych z pomocą funkcji [filter](https://docs.python.org/3/library/functions.html#filter).

In [ ]:
l = range(100000)

def isPower10(n):
    return (n ** 0.1).is_integer()

display([x for x in l if isPower10(x)])
display(filter(isPower10, l))
display(filter(lambda x: (x ** 0.1).is_integer(), l))

Odpowiednik powyższego dla `numpy`.

In [ ]:
import numpy as np

l = np.arange(100000)
l[(l) ** 0.1 % 1 == 0]

## Generatory i iteratory

### Problem:

Przetwarzanie ogromnych zbiorów danych może być kosztowne pod względem pamięci. Jeżeli nasz program operuje jedynie na drobnej części tych danych na raz a kolejność ich wykorzystania jest deterministyczna, możemy, zamiast przechowywać wszystko na raz, pozyskiwać dane na bieżąco. Jak odroczyć pozyskanie danych do momentu ich wykorzystania? Jak przechodzić po takich danych z pomocą pętli lub operacji na kolekcjach?

### Podejście [gorliwe](https://en.wikipedia.org/wiki/Evaluation_strategy#Eager_evaluation) a [leniwe](https://en.wikipedia.org/wiki/Lazy_evaluation)

Podstawowym podejściemy przy uzyskiwaniu danych jest tak zwane podejście gorliwe, to jest, uzyskujemy dane w momencie, gdy je definiujemy. Przykładowo, poniższy kod pokazuje, że lista tworzona z pomocą pętli tworzy się natychmiastowo i zajmuje odpowiednio duży obszar pamięci.

In [ ]:
from math import sqrt
from sys import getsizeof
l = []
for i in range(1000000):
    l.append(sqrt(i))

getsizeof(l)

Zamiast od razu wyznaczać wartości, możemy przygotować instrukcję, jak je otrzymać a samą procedurę tego wykonać dopiero gdy będziemy tego potrzebowali.

In [ ]:
from sys import getsizeof

N = 10 ** 15
getsizeof(zip(range(N), range(N)))

Mimo, że `l` powinna składać się z biliarda ($10^{15}$) elementów, kod ten wykonuje się natychmiastowo, a sam obiekt `l` zajmuje zaledwie $56$ B. Podejście takie nazywamy leniwym.

Operacje takie, jak [all](https://docs.python.org/3/library/functions.html#all), [any](https://docs.python.org/3/library/functions.html#any), [filter](https://docs.python.org/3/library/functions.html#filter), [map](https://docs.python.org/3/library/functions.html#map), [zip](https://docs.python.org/3/library/functions.html#zip) czy inne są właśnie funkcjami leniwymi.

Poniższy kod pokazuje, że `map` wykonuje się leniwie i dopiero gdy wartości są potrzebne. Z drugiej strony `any` spowodował zewaluowanie (pobranie wartości) dla całej tablicy. Czemu więc wciąż nazywamy funkcje `any` i `all` leniwymi?

In [ ]:
map(print, range(200))

In [ ]:
any(map(lambda x: print(x, end=" "), range(200)))

`zip` również nie wykonuje się a wyłącznie zwraca obiekt, który ma zostać wykonany w przyszłości.

In [ ]:
zip(map(lambda x: x ** 1.3 - x ** 1.2 + x, range(1000000)), range(1000000))

### Nieskończone kolekcje z [itertools](https://docs.python.org/3/library/itertools.html)

Czasami potrzebujemy szukać pewnych wartości na nieokreślenie długich zakresach. Może być wtedy przydatne stworzenie nieskończonej kolekcji. Moduł `itertools` udostępnia nam wiele takich funkcjonalności, jak [count](https://docs.python.org/3/library/itertools.html#itertools.count), [cycle](https://docs.python.org/3/library/itertools.html#itertools.cycle) czy [repeat](https://docs.python.org/3/library/itertools.html#itertools.repeat).

### [Iteratory](https://wiki.python.org/moin/Iterator)

Leniwe podejście przedstawione powyżej daje możliwość odroczenia pozyskania wartości. Możemy jednak również chcieć czasami nie tylko odraczać obliczanie wartości ale również nie przechowywać ich po wykorzystaniu.

Dla przykładu, $n$-tą liczbą Fibonacciego $F_n$ nazywamy taką liczbę, że $F_n = F_{n - 1} + F_{n - 2}$ i $F_0 = 0, F_1 = 1$. Oznacza to, że po wyznaczeniu iluś liczb Fibonacciego, do wyznaczenia kolejnej potrzebujemy wyłącznie dwóch poprzednich. Do tego typu zadań możemy wykorzystać tak zwane iteratory, to jest klasy, które udostępniają metody `__iter__` i `__next__`.

Załóżmy, że chcemy dostarczyć funkcjonalność zwracającą listę liczb Fibonacciego, a użytkownik chce, na przykład, policzyć sumę $n$ pierwszych. Prostym rozwiązaniem wtedy będzie:

In [ ]:
def fibs(n):
    l = [0, 1]
    for i in range(2, n):
        l.append(l[-1] + l[-2])
    return l

print(fibs(10))
print(sum(fibs(10)))

Aby nie tworzyć całej wielkiej tablicy zajmującej mnóstwo pamięci, możemy stwożyć tak zwany iterator, który pozwoli nam na wykonywanie operacji, tak, jakby tablica istniała, zarazem przechowując wyłącznie kilka wartości na raz.

In [ ]:
class FibonacciIterator:
    def __init__(self, n):
        self.n = n
        self.a = 0
        self.b = 1

    def __iter__(self):
        return self

    def __next__(self):
        self.n -= 1
        if self.n < 0:
            raise StopIteration
        else:
            self.a, self.b = self.a + self.b, self.a
            return self.b


iter = FibonacciIterator(100)
for elem in iter:
    print(elem, end=' ')

Teraz możemy na przykład stworzyć iterator Fibonacciego i pobrać z niego wyłącznie elementy będące podzielne przez $137$ i wyświetlić, na przykład, logarytm dziesiętny z logarytmu dziesiętnego z tej liczby powiększonej o $11$.

###### Logarytm to operacja będąca odwrotnością potęgowania. To jest, $\log_a b = c \iff a^c = b$
###### Logarytm dziesiętny to $\log_{10}$

###### <font color="gray">Ten kod jest czysto przykładowy a sam ciąg nie ma jakiegoś większego ukrytego sensu.</font>

In [ ]:
from math import log10

for f in map(lambda x: log10(x + 11), filter(lambda x: x % 137 == 0, FibonacciIterator(2000))):
    print(f)

### [Generatory](https://wiki.python.org/moin/Generators)

Analogiczne działanie możemy uzyskać za pomocą tak zwanych generatorów. Zamiast tworzyć całą klasę z polami `__iter__` i `__next__` możemy skorzystać ze słowa kluczowego [yield](https://docs.python.org/3/reference/expressions.html#yield-expressions).

Obszerniej w teorii programowania słowo-klucz `yield` zachowuje się jak `return` i zwraca wartość kończąc wywołanie funkcji. Przy kolejnym wywołaniu funkcji jednak ta kontynuuje działanie po tym `yield` właśnie.

W Pythonie `yield` jest leniwy, co oznacza, że zamiast wyniku zwraca wyłącznie instrukcję jego wykonania i iteracji.

In [ ]:
def f():
    print("First execution")
    yield 1
    print("Second execution")
    yield 2

for i in f():
    print(i)

f()

Ciąg Fibonacciego zrobiony z pomocą generatora.

In [ ]:
def fib():
    a, b = 1, 0
    while True:
        yield b
        a, b = a + b, a

l = zip(range(100), fib())
for (i, f) in l:
    print(f, end=' ')

## Wyrażenia regularne

### Problem:

Często przeszukując tekst musimy znaleźć wystąpienia pewnych podciągów spełniających pewien wzorzec. Kiedy indziej musimy wyodrębnić dane (data, numer serii, nazwa zbioru...) z nazw lub nagłówków plików.
Jak możemy znaleźć wystąpienia pewnego wzorca w tekście? Jak wyodrębnić te wystąpienia? Jak odczytać numer zawarty w nazwie pliku?

### [Wyrażenie regularne](https://en.wikipedia.org/wiki/Regular_expression) lub regex jest tekstem opisującym cechy spełniane przez inny tekst.

Przykładowo `"(Ala ma kota|Sierotka ma rysia).*"` oznacza, że opisany tekst będzie zaczynał się od `"Ala ma kota"` albo od `"Sierotka ma rysia"`.

Instrukcje tworzenia oraz interaktywny tester wyrażeń regularnych są udostępnione przez [RegExr](https://regexr.com/).

### Pakiet [re](https://docs.python.org/3/library/re.html)

Poniższy przykład został wygenerowany przez [asystenta OpenAI](https://chat.openai.com/chat) w odpowiedzi na pytanie `do you have some nice examples of package re in python?`.

In [ ]:
# This code is an example of how to use the re package in Python.
# The code was provided by Assistant, a large language model trained by OpenAI.

import re

# Find all occurrences of a pattern in a string
string = "The quick brown fox jumps over the lazy dog."
pattern = "the"
result = re.findall(pattern, string, flags=re.IGNORECASE)
print(result)
# Output: ['The', 'the', 'the']

# Replace all occurrences of a pattern in a string
string = "The quick brown fox jumps over the lazy dog."
pattern = "the"
replacement = "THE"
result = re.sub(pattern, replacement, string, flags=re.IGNORECASE)
print(result)
# Output: "THE quick brown fox jumps over THE lazy dog."

# Split a string by a pattern
string = "The quick brown fox jumps over the lazy dog."
pattern = "the"
result = re.split(pattern, string, flags=re.IGNORECASE)
print(result)
# Output: ['The quick brown fox jumps over ', ' lazy dog.']

# Check if a string matches a pattern
string = "The quick brown fox jumps over the lazy dog."
pattern = "the quick brown Fox jumps over the lazy Dog."
result = re.fullmatch(pattern, string, flags=re.IGNORECASE)
print(result)
# Output: <re.Match object; span=(0, 44), match='The quick brown fox jumps over the lazy dog.'>

In [ ]:
text = "Tomasz W. (33 l.), widziany ostatnio w Krakowie"
pattern = r"([A-Z]{1}[a-z]+ [A-Z]{1}\.) \((\d+) l.\)"
match = re.search(pattern, text)
print(match)
print(match.groups())
print(match.group(0))
print(match.group(1))
print(match.group(2))

text = "Tomasz (33 l.) i Ewa (24 l.) umówili się na jutro na wspólne zakupy"
pattern = r"([A-Z]{1}[a-z]+) \((\d+) l.\)"
print(re.findall(pattern, text))

### Elementy wzorców

#### Grupy

Jeżeli chcemy przechwycić konkretną część tekstu, otoczonego przez inny tekst, używamy okrągłych nawiasów (`( )`). W przypadku, gdy chcemy, by grupa była wykorzystana do tworzenia wzorca, ale nie była rozpatrywana jako przechwycona dodajemy za otwierającym nawiasem `?:`.

#### Kwantyfikatory i alteracje

Żeby zaznaczyć, że wzorzec ma wystąpić `n` razy używamy nawiasów klamrowych (`{ }`) z `n` w środku (`{n}`). Gdy chcemy, żeby dopasowało co najmniej `n` razy dodajemy po `n` przecinek (`{n,}`). Gdy chcemy by wystąpienia były pomiędzy `n` a `m` razy, rozdzielamy je przecinkiem (`{n,m}`).

Możemy również napisać po wzorcu `*` gdy chcemy, żeby wzorzec wystąpił dowolną ilość razy, `+` gdy chcemy, żeby wzorzec wystąpił co najmniej raz oraz `?` gdy chcemy, żeby wzorzec wystąpił najwyżej raz.

Możemy przekazać dwa wzorce do spełnienia jako alternatywy rozdzielając je `|`.

#### Ukotwiczenia

#### Klasy znaków

#### Przeszukiwanie

Wykorzystajmy pakiet `re` do sprawdzenia ile razy w Bibli ([World English Bible](https://github.com/scrollmapper/bible_databases)) występują słowa `hate`, `kill` i `love`.

In [ ]:
%pip install freebible
clear_output()

from freebible import read_web
import re

In [ ]:
web = read_web()
text = ''.join(
    [''.join(
        [''.join(
            [str(verse) for verse in chapter]
            ) for chapter in web[key]]
        ) for key in web.keys()]
    )
len(text)

In [ ]:
print(len(re.findall(r"[ \.,;\?!]hate[ \.,;\?!]", text, flags=re.IGNORECASE)))
print(len(re.findall(r"[ \.,;\?!]kill[ \.,;\?!]", text, flags=re.IGNORECASE)))
print(len(re.findall(r"[ \.,;\?!]love[ \.,;\?!]", text, flags=re.IGNORECASE)))

Policzmy też, jak często występują zdania o różnych długościach.

In [ ]:
sentences = re.findall("[A-Z](?:\w*[,;]? )*\w*[\.!\?]", text)
lengths = [len(re.findall("\w*", sentence)) for sentence in sentences]
buckets = [0] * (max(lengths) + 1)
for length in lengths:
    buckets[length] += 1

from matplotlib import pyplot as plt
plt.plot(buckets)

Czy to poprawna metoda?

## Profilowanie kodu

### Problem:

Często bardzo mała część kodu odpowiada za znaczną część czasu wykonania. W jaki sposób możemy zmierzyć czas i wydajność kodu? Jak powinniśmy porównywać wydajność?

### Przykładowe biblioteki do profilowania kodu

Odpowiedź [asystenta OpenAI](https://chat.openai.com/chat) na pytanie `what method of code profiling would you recommend for Python?`.

There are several methods of code profiling that can be used with Python, and the best method to use will depend on your specific goals and needs. Some common methods of code profiling in Python include:

 * The [timeit](https://docs.python.org/3/library/timeit.html) module, which can be used to measure the execution time of small pieces of code
 * The [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile) module, which provides a detailed breakdown of the execution time of a program
 * The [line_profiler](https://github.com/pyutils/line_profiler) and [memory_profiler](https://github.com/pythonprofilers/memory_profiler) modules, which can be used to profile the execution time and memory usage of specific lines of code

Each of these methods has its own strengths and limitations, and the best method to use will depend on the specific goals and needs of your project. Here is some more information about each of these methods:

 * The `timeit` module is a simple and easy-to-use method of profiling the execution time of small pieces of code. It provides a `timeit.timeit()` function that can be used to measure the time it takes for a given piece of code to run a specified number of times. This method is best suited for measuring the execution time of small, isolated pieces of code.

 * The `cProfile` module provides a more detailed and comprehensive view of the execution time of a Python program. It provides a `cProfile.run()` function that can be used to profile a Python program and generate a report that includes information about the execution time of each function in the program. This method is best suited for profiling larger, more complex programs and understanding the performance characteristics of different parts of the code.

 * The `line_profiler` and `memory_profiler` modules provide more specialized methods of profiling the execution time and memory usage of specific lines of code. These modules can be used to profile a Python program and generate a report that includes information about the execution time and memory usage of individual lines of code. This method is best suited for identifying and optimizing the performance of specific parts of a program.

### timeit

In [ ]:
import timeit

setup = "from math import sqrt"

code = '''
def func():
    return [sqrt(x) for x in range(1000000000000000000000)]
'''

print(timeit.timeit(stmt=code, setup=setup, number=100))

Czemu powyższy kod działa tak szybko?

In [ ]:
setup = "from math import sqrt"

code = '''
[sqrt(x) for x in range(10000)]
'''

print(timeit.timeit(stmt=code, setup=setup, number=1000))

Po wykonaniu kodu z `setup` `timeit.timeit` wywołuje kod wprowadzony w `stmt` `number` powtórzeń.

Analogicznie możemy wykonać własnoręcznie pobierając aktualny czas z `timeit.default_timer`.

In [ ]:
from math import sqrt

start = timeit.default_timer()
for i in range(1000000):
    [sqrt(x) for x in range(100)]
print(timeit.default_timer() - start)

In [ ]:
%timeit [sqrt(x) for x in range(10000)]

### cProfile

Kod wygenerowany przez [asystenta OpenAI](https://chat.openai.com/chat)

In [ ]:
# This code is an example of how to use the re package in Python.
# The code was provided by Assistant, a large language model trained by OpenAI.

import cProfile

def add_lists(a, b):
    c = []
    for i in range(len(a)):
        c.append(a[i] + b[i])
    return c

def main():
    a = [1, 2, 3, 4, 5]
    b = [6, 7, 8, 9, 10]
    c = add_lists(a, b)

# Profile the execution of the main() function
cProfile.run('main()')


### Graficzna reprezentacja wydajności

Do wyświetlania wykresów wykorzystujemy bibliotekę [matplotlib](https://matplotlib.org/).

In [ ]:
from matplotlib import pyplot as plt

plt.plot([1, 5, 2, 3])

In [ ]:
from random import random
from timeit import default_timer
N = 1000
times = []
ns = range(N, N * 100, N)
for n in ns:
    l = [random() for _ in range(n)]
    start = default_timer()
    l.sort()
    times.append(default_timer() - start)

plt.plot(list(ns), times)


## Wielowątkowość

### Problem:

Czas wykonania kodu może być ograniczony przez trzy czynniki: przetwarzanie danych, pobieranie danych oraz zewnętrzne systemy. Jeżeli jest to kwestia zewnętrznych systemów, to nasz program jest nieaktywny. Możemy w tym czasie wykonywać inne zadania. W jaki sposób możemy uruchomić zadanie w czasie, gdy kod czeka na otrzymanie wyniku z innego? W jaki sposób możemy wykonać kilka zadań w tym samym czasie? W jaki sposób możemy podzielić wykonanie zadania na kilka równoległych?

### Wątek

Program jest [procesem](https://en.wikipedia.org/wiki/Process_(computing)) z ewentualnym zestawem podprocesów. Każdy proces posiada swój zarezerwowany obszar pamięci oraz swój proces-rodzica, który nim zarządza. Tworzenie oddzielnych procesów jest kosztowne, dla małych, równoległych operacji możemy zamiast tego wykorzystać tak zwane [wątki](https://en.wikipedia.org/wiki/Thread_(computing)). Są to lekkie, równolegle wykonujące się zestawy operacji, które korzystają z tej samej przestrzeni pamięci.

Do tworzenia i zarządzania wątkami w Pythonie korzystamy z biblioteki [threading](https://docs.python.org/3/library/threading.html).

#### [threading.start](https://docs.python.org/3/library/threading.html#threading.Thread.start)

In [ ]:
import threading
from time import sleep
from random import random

def countTo10():
    for i in range(10):
        sleep(random() * 0.1)
        print(i, end='')

t1 = threading.Thread(target=countTo10)
t2 = threading.Thread(target=countTo10)
t1.start()
t2.start()
print("Am I finished?")

#### [threading.join](https://docs.python.org/3/library/threading.html#threading.Thread.join)

In [ ]:
import threading
from time import sleep
from random import random

def countTo10():
    for i in range(10):
        sleep(random() * 1)
        print(i, end=' ')

def sleep6s():
    print("I start my sleep")
    sleep(3)
    print("I go asleep again")
    sleep(3)

t1 = threading.Thread(target=countTo10)
t2 = threading.Thread(target=sleep6s)
t1.start()
t2.start()
print("Am I finished?")
t1.join()
t2.join()

#### Wątki z argumentami

In [ ]:
import threading
from time import sleep
from random import random

def countToN(n):
    for i in range(n):
        sleep(random() * 0.1)
        print(i, end='')

t1 = threading.Thread(target=countToN, args=(5,))
t2 = threading.Thread(target=countToN, args=(10,))
t1.start()
t2.start()
print("Am I finished?")
t1.join()
t2.join()

In [ ]:
import threading
from time import sleep
from random import random

def greet(k, n):
    for i in range(n):
        sleep(random() * 0.1)
        print(f"{k}: Hello {i}th time")

t1 = threading.Thread(target=greet, args=(0, 50))
t2 = threading.Thread(target=greet, args=(1, 50))
t1.start()
t2.start()
print("Am I finished?")
t1.join()
t2.join()

#### [Wyścig](https://en.wikipedia.org/wiki/Race_condition)

In [ ]:
import threading
from time import sleep
from random import random

sharedCounter = 0
def add1000000():
    global sharedCounter
    for i in range(1000000):
        sharedCounter += 1

t1 = threading.Thread(target=add1000000)
t2 = threading.Thread(target=add1000000)
t1.start()
t2.start()
t1.join()
t2.join()
print(sharedCounter)

#### [Lock](https://docs.python.org/3/library/threading.html#threading.Lock)

In [ ]:
import threading
from time import sleep
from random import random

lock = threading.Lock()
sharedCounter = 0

def add1():
    global sharedCounter
    lock.acquire()
    sharedCounter += 1
    lock.release()

def add1000000():
    global sharedCounter
    for i in range(1000000):
        add1()

t1 = threading.Thread(target=add1000000)
t2 = threading.Thread(target=add1000000)
t1.start()
t2.start()
t1.join()
t2.join()
print(sharedCounter)

#### Zwracanie wartości przez wątek

In [ ]:
import threading
from random import random

N = 10000
M = 100
results = [None] * M


arr = [[random() for _ in range(N)] for _ in range(N)]

def calculateSum(arr, n, k, results):
    results[n] = sum([sum(row) for row in arr[n * k: n * k + k]])

threads = [threading.Thread(target=calculateSum, args=(arr, i, N // M, results)) for i in range(M)]
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

sum(results)

### [Global Interpreter Lock](https://en.wikipedia.org/wiki/Global_interpreter_lock)

Zmierzmy czas wykonania tego wywołania dla wielu wątków.

In [ ]:
import threading
from random import random

N = 10000
M = 100
results = [None] * M


arr = [[random() for _ in range(N)] for _ in range(N)]

def calculateSum(arr, n, k, results):
    results[n] = sum([sum(row) for row in arr[n * k: n * k + k]])


In [ ]:
%%timeit
threads = [threading.Thread(target=calculateSum, args=[arr, i, N // M, results]) for i in range(M)]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

Zmierzmy również czas wykonania dla jednego wątku.

In [ ]:
result = [None]

%timeit calculateSum(arr, 0, N, result)

Czemu te czasy są podobne? Python nie pozwala na rzeczywistą wielowątkowość zezwalając na działanie wyłącznie jednego wątku na raz. Zaimplementowane jest z pomocą tak zwanego Globalnej Blokady Interpretatora. Jeżeli jeden wątek korzysta z danych i coś na nich wykonuje wszystkie pozostałe czekają na zwolnienie tych zasobów.

### Multiprocessing

Zamiast tworzyć wątki w ramach naszego procesu możemy stworzyć oddzielny podproces. Procesy posiadają własną przestrzeń pamięci i działają niezależnie od siebie. Nie współdzielą domyślnie żadnych zmiennych i nie ma potrzeby tworzenia blokad na obiektach jak w przypadku wątków. Zaletą jest możliwość wykonywania ich równolegle na kilku jądrach procesora, wadą zaś, droga procedura tworzenia oraz droga procedura komunikowania się pomiędzy procesami.

Biblioteka [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) udostępnia nam funkcjonalności takie jak:

 * [Process](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process) - klasa pojedynczego procesu, analogiczna do `Thread`.
 * [Pool](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) - klasa zarządzająca wieloma procesami i rozdzielająca zadania pomiędzy nie. Działa dla funkcjonalności iterowania po listach.
 * [Manager](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Manager) - pozwala na współdzielenie obiektów pomiędzy procesami.
